In [ ]:
## 파이널 컷에서 wav 파일 추출까지 완료 한 상태에서 아래 시작

In [39]:
import torch

In [40]:
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    mps_device = torch.device("mps")

    # Create a Tensor directly on the mps device
    x = torch.ones(5, device=mps_device)
    # Or
    x = torch.ones(5, device="mps")

    # Any operation happens on the GPU
    y = x * 2


In [1]:
import whisper
import warnings
warnings.filterwarnings('ignore')
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_silence


In [52]:
model = whisper.load_model('large-v2')
result = model.transcribe('New Recording 28.m4a', language = 'ko')
print(result['text'])

 대구가 보고 기간 말 현재 창고에 보관 중인 재고 자산은 10만 원이다. 다음 자료를 이용하여 대구의 기초 재고 자산을 계산하면 얼마인가?


In [60]:
# wav 주소 가져오기
target_audio = 'New Recording 28.m4a' # 주소

# 오디오 파일 불러오기
audio = AudioSegment.from_file(target_audio, format="m4a")#wav

min_silence_len = 350  # 무음으로 간주될 최소 길이 (밀리초 단위) ----> 녹음 환경 및 소음에 따라 적절하게 조정
silence_thresh = -35 # 무음으로 간주될 데시벨 값 -----> 녹음 환경 및 소음에 따라 적절하게 조정

# 무음 부분을 기준으로 오디오 분할
chunks = split_on_silence(
    audio,
    min_silence_len=min_silence_len,  # 무음으로 간주될 최소 길이 (밀리초 단위)
    silence_thresh=silence_thresh,  # 무음으로 간주될 데시벨 값
    keep_silence=0,
)
print(len(chunks))

silences = detect_silence(
    audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh
)
print(silences[:10])
silences_diff = [s[1] - s[0] for s in silences]

6
[[0, 1474], [4594, 5189], [5736, 7346], [7645, 8583], [9725, 10543], [12682, 13259], [13542, 15723]]


In [61]:
## 분리된 녹음 합치고 들어보기

output = chunks[0]
output += audio[silences[0][0] : silences[0][1]]
for i in range(1, 3):
    output += chunks[i]
    output += audio[silences[i][0] : silences[i][1]]

output

In [62]:
from tqdm import tqdm

# 분할된 각 청크를 파일로 저장 (예시)
current_duration = 0.0
timeline = []
transcripts = []

# 사용할 whisper model 불러오기
model = whisper.load_model('large-v2')

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    f_name = f"sample/chunk{i}.wav"
    chunk.export(f_name, format="wav")
    try:
        transcript = model.transcribe(f_name, language ='ko') 
        
    except:
        transcript = ""

    start = current_duration
    if i < len(silences_diff):
        end = current_duration + chunk.duration_seconds * 1000 + silences_diff[i]
    else:
        end = current_duration + chunk.duration_seconds * 1000

    print(int(start), int(end))
    timeline.append((start, end))
    current_duration = end
    transcripts.append(transcript)

 17%|█▋        | 1/6 [00:07<00:39,  7.98s/it]

0 4594


 33%|███▎      | 2/6 [00:13<00:25,  6.50s/it]

4594 5736


 50%|█████     | 3/6 [00:18<00:17,  5.90s/it]

...

7645 9725


 83%|████████▎ | 5/6 [00:31<00:06,  6.41s/it]

9725 12682


100%|██████████| 6/6 [00:37<00:00,  6.21s/it]

12682 13542


In [37]:
def format_time(ms):
    """밀리초를 SRT 포맷의 시간 문자열로 변환합니다."""
    seconds, milliseconds = divmod(ms, 1000)
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"


def create_srt(chunks, subtitles, filename):
    """SRT 파일을 생성합니다."""
    with open(filename, "w") as file:
        combined = [
            (start, end, text) for (start, end), text in zip(timeline, transcripts)
        ]
        for i, (start, end, text) in enumerate(combined, start=1):
            file.write(f"{i}\n")
            file.write(f"{format_time(start)} --> {format_time(end)}\n")
            file.write(f"{text}\n\n")


# SRT 파일 생성
create_srt(timeline, transcripts, "subtitles.srt")